In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00


In [2]:
import logging
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from sklearn.metrics import accuracy_score, f1_score

# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls /content/drive/MyDrive/Ind_Proj/large_language_models

LLama_2_7b_chat_multidataset_finetuned	Sentences_AllAgree.txt


In [6]:
!cp -r /content/drive/MyDrive/Ind_Proj/large_language_models/Sentences_AllAgree.txt /content/

In [7]:
!ls /content/

drive  sample_data  Sentences_AllAgree.txt


In [8]:
# Define a custom loading function
def process_line(line):
    # Split the line by the @ symbol
    text, label = line.rsplit('@', 1)
    return {"text": text.strip(), "label": label.strip()}

# Load the dataset from your text file
dataset = load_dataset('text', data_files='Sentences_AllAgree.txt',split='train', encoding='ISO-8859-1')

# Map the text file lines to 'text' and 'label' features
dataset = dataset.map(lambda example: process_line(example['text']))

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2264 [00:00<?, ? examples/s]

In [24]:
# Shuffle the dataset and select the first 1000 rows
fpb_dataset = dataset.shuffle(seed=42).select(range(1000))

# Check the new dataset
print(fpb_dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})


In [10]:
model_name = "NousResearch/Llama-2-7b-chat-hf"

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [11]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

In [12]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

Your GPU supports bfloat16: accelerate training with bf16=True


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [13]:
# Define the text-generation pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

In [14]:
# Function to generate sentiment
def generate_sentiment(text):
    # Construct the prompt
    prompt = f"Determine the sentiment of the financial news as negative, neutral or positive: {text}"

    # Run the text generation
    result = pipe(f"<s>[INST] {prompt} [/INST]")

    # Extract the generated text
    generated_text = result[0]['generated_text']

    # Extract the text after [/INST]
    return extract_after_inst(generated_text)

In [15]:
# Function to extract the part after [/INST]
def extract_after_inst(generated_text):
    parts = generated_text.split('[/INST]')
    return parts[1].strip() if len(parts) > 1 else generated_text

In [16]:
# Function to check sentiment in the model's output
def check_sentiment(output):
    output = output.lower()
    if "negative" in output:
        return "negative"
    elif "neutral" in output:
        return "neutral"
    elif "positive" in output:
        return "positive"
    else:
        return "neutral"  # Default to "neutral" if no keyword found

In [25]:
def generate_and_classify_batch(batch):
    texts = batch['text']
    actual_labels = batch['label']

    # Generate predictions for the batch
    generated_texts = pipe([f"<s>[INST] Determine the sentiment of the financial news as negative, neutral or positive: {text} [/INST]" for text in texts])

    # Flattening the nested lists of generated texts
    if isinstance(generated_texts[0], list):
        generated_texts = [item for sublist in generated_texts for item in sublist]

    # Extract and classify sentiment from generated texts
    predicted_sentiments = [check_sentiment(extract_after_inst(result['generated_text'])) for result in generated_texts]

    print("Batch completed.")

    return {'y_true': actual_labels, 'y_pred': predicted_sentiments}

# Apply the transformation using map with batch_size
batch_size = 64
result = fpb_dataset.map(generate_and_classify_batch, batched=True, batch_size=batch_size)

# Extract true and predicted labels for evaluation
y_true = result['y_true']
y_pred = result['y_pred']

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.


In [26]:
# Evaluate the model using accuracy and F1 score
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted')

# Print the evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.5560
F1 Score: 0.5912


In [28]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Determine the sentiment of the financial news as negative, neutral or positive:Consumer credit $18.9BN, Exp. $16BN, Last $9.6BN."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Determine the sentiment of the financial news as negative, neutral or positive:Consumer credit $18.9BN, Exp. $16BN, Last $9.6BN. [/INST]  Based on the information provided, the sentiment of the financial news can be classified as neutral.

The news reports a decrease in consumer credit compared to the expected amount ($18.9BN vs $16BN) and a decrease in the last reported amount ($9.6BN) compared to the previous period. This suggests that consumer credit is not growing as rapidly as expected or as it was in the previous period, which could be interpreted as a neutral or even negative development.

However, it's important to note that the news does not provide any context or additional information that could indicate a more positive or negative sentiment. Therefore, a neutral classification seems to be the most appropriate based on the information
